In [ ]:
pip install xarray zarr gcsfs fsspec pandas geopandas rasterstats

- xarray: For handling Zarr datasets.
- zarr and gcsfs: For accessing Zarr files on GCS.
- fsspec: For file system abstraction.

## Google Cloud data subset maintained by ARCO-ERA5 at Google (lagged by three months)

In [ ]:
import xarray as xr
import gcsfs
import pandas as pd

In [ ]:
# Initialize GCS file system (no authentication needed for public data)
gcs = gcsfs.GCSFileSystem(token='anon')

# Specify the Zarr store (cloud-optimized single-level reanalysis)
zarr_path = 'gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr'

# Open the Zarr store with xarray
ds = xr.open_zarr(zarr_path, consolidated=True, storage_options={'token': 'anon'})



In [2]:
ds

<xarray.Dataset> Size: 4PB
Dimensions:                                                          (
                                                                      time: 1323648,
                                                                      latitude: 721,
                                                                      longitude: 1440,
                                                                      level: 37)
Coordinates:
  * latitude                                                         (latitude) float32 3kB ...
  * level                                                            (level) int64 296B ...
  * longitude                                                        (longitude) float32 6kB ...
  * time                                                             (time) datetime64[ns] 11MB ...
Data variables: (12/273)
    100m_u_component_of_wind                                         (time, latitude, longitude) float32 5TB ...
    100m_v_component_of_wind                                         (time, latitude, longitude) float32 5TB ...
    10m_u_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB ...
    10m_u_component_of_wind                                          (time, latitude, longitude) float32 5TB ...
    10m_v_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB ...
    10m_v_component_of_wind                                          (time, latitude, longitude) float32 5TB ...
    ...                                                               ...
    wave_spectral_directional_width_for_swell                        (time, latitude, longitude) float32 5TB ...
    wave_spectral_directional_width_for_wind_waves                   (time, latitude, longitude) float32 5TB ...
    wave_spectral_kurtosis                                           (time, latitude, longitude) float32 5TB ...
    wave_spectral_peakedness                                         (time, latitude, longitude) float32 5TB ...
    wave_spectral_skewness                                           (time, latitude, longitude) float32 5TB ...
    zero_degree_level                                                (time, latitude, longitude) float32 5TB ...
Attributes:
    valid_time_start:       1940-01-01
    last_updated:           2025-05-05 02:19:41.078356+00:00
    valid_time_stop:        2025-01-31
    valid_time_stop_era5t:  2025-04-29

In [3]:
import xarray as xr
import gcsfs

# 1) Point to the ARCO-ERA5 Analysis-Ready store
ar_zarr = "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3"

# 2) Open with anonymous GCS credentials
gcs = gcsfs.GCSFileSystem(token="anon")
ds = xr.open_zarr(
    ar_zarr,
    consolidated=True,
    storage_options={"token": "anon"}
)

# (Optional) Inspect dims & coords to confirm names
print(ds.dims)        # e.g. ('time','latitude','longitude','level')
print(ds.coords)      # to see lat/lon ranges
print(ds.data_vars)   # to see '2t' in the dataset

# 3) Define Illinois bounding box (approximate)
#    Latitude: 36.95°N (south) → 42.50°N (north)
#    Longitude: –91.5° (west) → –87.0° (east)
lat_min, lat_max = 36.95, 42.50
lon_min, lon_max = -91.50, -87.00

# 4) Slice out just the 2 m temperature ("2t") over IL
ds_il_temp = (
    ds["2m_temperature"]
      .sel(
        time     = slice(ds.attrs["valid_time_start"], ds.attrs["valid_time_stop"]),
        latitude = slice(lat_max, lat_min),   # slice(high, low) if coord is descending
        longitude= slice(lon_min, lon_max)
      )
      .load()  # actually pull the data
)

print(ds_il_temp)



FrozenMappingWarningOnValuesAccess({'time': 1323648, 'latitude': 721, 'longitude': 1440, 'level': 37})
Coordinates:
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) int64 296B 1 2 3 5 7 10 20 ... 875 900 925 950 975 1000
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 11MB 1900-01-01 ... 2050-12-31T23:00:00
Data variables:
    100m_u_component_of_wind                                         (time, latitude, longitude) float32 5TB ...
    100m_v_component_of_wind                                         (time, latitude, longitude) float32 5TB ...
    10m_u_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB ...
    10m_u_component_of_wind                                          (time, latitude, longitude) float32 5TB ...
    10m_v_component_of_neutral_wind                                  (time, latitude, longitude) float32

In [4]:
print(ds_il_temp)


<xarray.DataArray '2m_temperature' (time: 745872, latitude: 23, longitude: 0)> Size: 0B
array([], shape=(745872, 23, 0), dtype=float32)
Coordinates:
  * latitude   (latitude) float32 92B 42.5 42.25 42.0 41.75 ... 37.5 37.25 37.0
  * longitude  (longitude) float32 0B 
  * time       (time) datetime64[ns] 6MB 1940-01-01 ... 2025-01-31T23:00:00
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K


In [ ]:
# 5) Save to a local NetCDF for downstream use
ds_il_temp.to_netcdf("IL_2m_temperature_ERA5_AR_1979-present.nc")


In [ ]:
df = ds_il_temp.to_dataframe()
#df.to_csv('era5_subset.csv')
df

In [ ]:

# Convert units
ds_subset['2m_temperature'] = ds_subset['2m_temperature'] - 273.15  # Kelvin to Celsius
ds_subset['total_precipitation'] = ds_subset['total_precipitation'] * 1000  # Meters to mm

# Save to a local NetCDF file (optional, for smaller subsets)
ds_subset.to_netcdf('era5_subset.nc')